In [2]:
!pip -q install bitsandbytes accelerate xformers einops peft datasets
!pip -q install --upgrade huggingface_hub
!pip -q install PyPDF2 pycryptodome==3.15.0 sentencepiece

import warnings
warnings.filterwarnings('ignore')

In [3]:
import torch
import transformers
import re, random
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "mistralai/Mistral-7B-Instruct-v0.1" # nama model
# model di load dalam format 4-bit quantized
bnb_config = transformers.BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
)

login(token="token")

# load model
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        quantization_config=bnb_config,
        device_map='auto',
)

# load tokenizer model
tokenizer = transformers.AutoTokenizer.from_pretrained(
            model_id,
            padding_side="left",
            add_eos_token=True,
            add_bos_token=True
)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
!pip install PyPDF2

In [4]:
!curl https://ecss.nl/wp-content/uploads/standards/ecss-e/ECSS-E-ST-50-51C5February2010.pdf --output ECSS-E-ST-50-51C5February2010.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  177k  100  177k    0     0  92767      0  0:00:01  0:00:01 --:--:-- 92752


ekstrak data teks pada pdf

In [5]:
import PyPDF2
import re

def pdf_to_text(pdf_path, skip_start_pages=0, skip_last_pages=0, header_lines=1, footer_lines=1):
  with open(pdf_path, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    num_pages = len(pdf_reader.pages)

    for page_num in range(skip_start_pages - 1, num_pages-skip_last_pages):
      page = pdf_reader.pages[page_num]
      page_text = page.extract_text()

      lines = page_text.splitlines(True)[header_lines:-footer_lines] # Removing header and footer

      lines_modified = []
      for line in lines:

        # Optional pre-processing of lines to correct errors
        # None

        lines_modified.append(line)

      lines_joined = "".join(lines_modified)
      text += lines_joined
  return text

pdf_file_path = "/content/ECSS-E-ST-50-51C5February2010.pdf"
raw_text = pdf_to_text(pdf_file_path, skip_start_pages=5, skip_last_pages=0, header_lines=2, footer_lines=1) # skip_start_pages=5 diubah jadi 0
print(len(raw_text))

12990


ubah data teks hasil ekstrak pdf menjadi format Q&A

In [6]:
import json

def format_question_prompt(input):
  return f"""Below is a input text extracted from a document. Generate one question that is answered with the given text. Do not include any answer in the generated text.

### Input:
 {input}
"""

def format_answer_prompt(input,instruction):
  return f"""Below is an instruction that describes a task or a question, paired with an input that provides context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}
"""

def generate_response(prompt):
  chat = [
      {"role": "user", "content": prompt},
  ]
  text = tokenizer.apply_chat_template(chat, tokenize=False)
  encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False).to('cuda')
  generated_ids = model.generate(**encodeds, pad_token_id=tokenizer.eos_token_id, max_new_tokens=1024, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

  # Remove prompt and special characters
  cleaned_response = decoded.split(prompt, 1)[1]  # Split by prompt and keep the second part
  cleaned_response = tokenizer.clean_up_tokenization(cleaned_response)  # Remove special tokens
  cleaned_response = re.sub(r"\ \[/INST\] ", "", cleaned_response)  # Match and replace [/INST] followed by a space

  return cleaned_response

sample_char_len_ar = [1024]
num_interations = 20
data= []

for  sample_char_len in sample_char_len_ar:
    for i in range(num_interations):
        print(i)
        random_integer = random.randint(0, len(raw_text) - sample_char_len)
        document_sample = raw_text[random_integer:random_integer+sample_char_len]
        words = document_sample.split()
        words.pop(0)
        words.pop()
        document_sample = " ".join(words)
        document_sample = document_sample.replace("\u2010", " ")
        prompt = format_question_prompt(document_sample)
        instruction = generate_response(prompt)

        prompt = format_answer_prompt(document_sample, instruction)
        output = generate_response(prompt)

        data.append({
          "instruction": instruction,
          "input": ' '.join(document_sample.split()),
          "output": output
        })

with open("data.json", "w") as f:  # Open in write mode to overwrite
    json.dump(data, f, indent=4)  # Write the formatted data to the file

print("Data successfully written to JSON file.")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Data successfully written to JSON file.


In [8]:
import json
import random
from datasets import Dataset

with open("data.json", "r") as f:
   dataset = json.load(f)

In [9]:
len(dataset) # jumlah data

50

split data

In [10]:
if isinstance(dataset, list):

    random.shuffle(dataset)

    train_dataset_list = dataset[:40]
    eval_dataset_list = dataset[40:50]

    for item in train_dataset_list:
        item["input"] = ""

    for item in eval_dataset_list:
        item["input"] = ""

    with open("train_data.json", "w") as f:
        json.dump(train_dataset_list, f)

    with open("eval_data.json", "w") as f:
        json.dump(eval_dataset_list, f)

else:
    print("Data is not a list, cannot shuffle.")

In [11]:
import json
from datasets import Dataset

def format_prompt_user(sample):
    return f"""
Below is an instruction that describes a task or a question, paired with an optional input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{sample["instruction"]}

### Input:
{sample["input"]}
"""

def format_prompt_assistant(sample):
    return f"""
### Response:
{sample["output"]}
"""

def format_prompt_chat(sample):
    chat = [
        {"role": "user", "content": format_prompt_user(sample)},
        {"role": "assistant", "content": format_prompt_assistant(sample)}
    ]
    text = tokenizer.apply_chat_template(chat, tokenize=False)
    return {"formatted_text": text}

def generate_and_tokenize_prompt(sample):
    formatted_text = sample["formatted_text"]
    result = tokenizer(
        formatted_text,
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

max_length = 2048  # differs from datasets to datasets

with open("train_data.json", "r") as f:
    train_dataset_list = json.load(f)

with open("eval_data.json", "r") as f:
    eval_dataset_list = json.load(f)

train_dataset = Dataset.from_list(train_dataset_list)
eval_dataset = Dataset.from_list(eval_dataset_list)

formatted_train_dataset = train_dataset.map(lambda x: format_prompt_chat(x))
formatted_eval_dataset = eval_dataset.map(lambda x: format_prompt_chat(x))

tokenized_train_dataset = formatted_train_dataset.map(generate_and_tokenize_prompt)
tokenized_eval_dataset = formatted_eval_dataset.map(generate_and_tokenize_prompt)

print(f'Training data size: {len(tokenized_train_dataset)}')
print(f'Validation data size: {len(tokenized_eval_dataset)}')

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Training data size: 40
Validation data size: 10


In [6]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [13]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from datetime import datetime

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

project = "ECSS-E-ST-50-51C-finetune-test-#3"
base_model_name = "Mistral-7B-Instruct"
run_name = base_model_name + "-" + project
output_dir = "/content/" + run_name

# Preapre model
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=8,
        lora_alpha=64,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "lm_head"]
    )
model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=2,
        gradient_checkpointing=True,
        max_steps=2,
        learning_rate=2.5e-5, # Want about 10x smaller than the Mistral learning rate
        logging_steps=1,
        bf16=True,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=1,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=1,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        #report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

# Modify the compute_loss function of the Trainer
def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
    """
    How the loss is computed by Trainer. By default, all models return the loss in the first element.
    Subclass and override for custom behavior.
    """
    labels = inputs.get("labels")
    # forward pass
    outputs = model(**inputs)
    # Save past state if it exists
    # TODO: this needs to be fixed and made cleaner later.
    if self.args.past_index >= 0:
        self._past = outputs[self.args.past_index]

    # compute custom loss (suppose one has 3 labels with different weights)
    loss = outputs.loss  # Assuming the model returns the loss in the 'loss' attribute

    return (loss, outputs) if return_outputs else loss

trainer.compute_loss = compute_loss.__get__(trainer) # Bind the method to the trainer instance

if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

#trainer.train(resume_from_checkpoint = True)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


trainable params: 21260288 || all params: 3773331456 || trainable%: 0.5634354746703705


Step,Training Loss,Validation Loss
1,5.131500,2.837119
2,5.339600,2.789421


TrainOutput(global_step=2, training_loss=5.235524654388428, metrics={'train_runtime': 505.1834, 'train_samples_per_second': 0.008, 'train_steps_per_second': 0.004, 'total_flos': 350548150714368.0, 'train_loss': 5.235524654388428, 'epoch': 0.1})

inference

In [15]:
import torch
import transformers
import re, random
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, PeftModel, PeftConfig, prepare_model_for_kbit_training, get_peft_model

model_id = "mistralai/Mistral-7B-Instruct-v0.1"

project = "ECSS-E-ST-50-51C-finetune-test-#3"
base_model_name = "Mistral-7B-Instruct"
run_name = base_model_name + "-" + project
output_dir = "/content/" + run_name
check_point = "checkpoint-2"
adapter_model_id = output_dir + "/" + check_point

bnb_config = transformers.BitsAndBytesConfig(
              load_in_4bit=True,
              bnb_4bit_use_double_quant=True,
              bnb_4bit_quant_type="nf4",
              bnb_4bit_compute_dtype=torch.bfloat16
)

config = PeftConfig.from_pretrained(adapter_model_id)

login(token="token hf")

model = transformers.AutoModelForCausalLM.from_pretrained(
          model_id,
          trust_remote_code=True,
          quantization_config=bnb_config,
          device_map='auto',
)

ft_model = PeftModel.from_pretrained(
          model,
          adapter_model_id,
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
            model_id,
            padding_side="left",
            add_eos_token=True,
            add_bos_token=True
)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ajukan pertanyaan

In [16]:
def generate_response(prompt):
  chat = [
      {"role": "user", "content": prompt},
  ]
  text = tokenizer.apply_chat_template(chat, tokenize=False)
  encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False).to('cuda')
  generated_ids = ft_model.generate(**encodeds, pad_token_id=tokenizer.eos_token_id, max_new_tokens=1024, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

  # Remove prompt and special characters
  cleaned_response = decoded.split(prompt, 1)[1]  # Split by prompt and keep the second part
  cleaned_response = tokenizer.clean_up_tokenization(cleaned_response)  # Remove special tokens
  cleaned_response = re.sub(r"\ \[/INST\] ", "", cleaned_response)  # Match and replace [/INST] followed by a space

  return cleaned_response

# Chat loop
while True:
  user_input = input("You: ")
  if user_input.lower() == "quit":
    break
  response = generate_response(user_input)
  print(f"Computer: {response}")

You: what content of the text
Computer: The content of the text is not specified as it is an open-ended prompt. Please provide more context or instructions to generate a specific text content.
You: explain to me about SpaceWire 
Computer: SpaceWire is a high-speed interconnect technology specifically designed for use in spacecraft and space applications. It was developed by the European Space Agency (ESA) and uses a unique, bidirectional protocol for efficiently transmitting data between different subsystems within a space vehicle.

One of the key advantages of SpaceWire is its ability to operate over a wide range of temperatures and environments, making it suitable for use in both unmanned spacecraft and manned space vehicles. It also features a high degree of fault tolerance and reliability, which is crucial in the demanding environment of space.

SpaceWire uses a combination of bus and ring topologies to transmit data between different subsystems within a space vehicle. It uses a un

sumber

https://medium.com/@ales.avanzi/effective-q-a-llm-fine-tuning-with-pdf-documentation-91b7fc458ced